## 神经网络快速入门

### 简单网络

In [ ]:
import numpy as np
from mpmath import sigmoid
from tensorboard.program import TensorBoardServer
from torch.utils.data import Dataset, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from urllib3.filepost import writer

x = np.random.randint(1, 100, 100)
y = np.array([i * 10 + np.random.randint(100) for i in x])

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x, y)  # 散点图
plt.show()

### 归一化

In [ ]:
x_norm = x / 100
y_norm = y / 100

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x_norm, y_norm)  # 散点图
plt.show()

#### min-max 归一化
1. 能保证数据在0-1之间
2. 能保持数据的分布
$$ x' = \frac{x - x_{min}}{x_{max} - x_{min}} $$

In [ ]:
x_min_max = (x - x.min()) / (x.max() - x.min())
y_min_max = (y - y.min()) / (y.max() - y.min())

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x_min_max, y_min_max)  # 散点图
plt.show()

#### Z-score 归一化
1，能保证数据的发布
2. 对于异常数据没那么敏感
$$ x' = \frac{x - \mu}{\sigma} $$

In [ ]:
x_z_score = (x - x.mean()) / x.std()
y_z_score = (y - y.mean()) / y.std()

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x_z_score, y_z_score)
plt.show()

### 构建网络

In [ ]:
w = 0.1
step = 5000
plt.scatter(x_z_score, y_z_score)
for i in range(step):
    for j in range(len(x_z_score)):
        y_predict = w * x_z_score[j]
        loss = y_predict - y_z_score[j]
        w -= loss * x_z_score[j]
y_predict = w * x_z_score
plt.plot(x_z_score, y_predict)

#### 梯度下降


In [ ]:
w = 0.1
step = 10
plt.scatter(x_min_max, y_min_max)
for _ in range(step):
    for i in range(len(x_min_max)):
        dloss = 2 * w * x_min_max[i] ** 2 - 2 * x_min_max[i] * y_min_max[i]
        w -= dloss
    print(w)
y_predict = w * x_min_max
plt.plot(x_min_max, y_predict)

#### 随机梯度下降

In [ ]:
w = 0.1
step = 200
alpha = 0.1
plt.scatter(x_min_max, y_min_max)
for _ in range(step):
    for i in range(np.random.randint(1, len(x_min_max)), 20):
        dloss = 2 * w * x_min_max[i] ** 2 - 2 * x_min_max[i] * y_min_max[i]
        w -= dloss * alpha
        print(w)
y_predict = w * x_min_max
plt.plot(x_min_max, y_predict)

#### pytorch 随机梯度下降

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

x = np.random.randint(1, 100, 100)
y = np.array([i * 10 + np.random.randint(100) for i in x])

x_min_max = (x - x.min()) / (x.max() - x.min())
y_min_max = (y - y.min()) / (y.max() - y.min())

x_torch = torch.tensor(x_min_max, dtype=torch.float32)
y_torch = torch.tensor(y_min_max, dtype=torch.float32)

w = torch.nn.Parameter(torch.rand(1, dtype=torch.float32), requires_grad=True)
optimizer = torch.optim.SGD([w], lr=0.01)  # w -= altha * loss

step = 200
for _ in range(step):
    for i in range(np.random.randint(1, len(x_min_max))):
        y_predict = w * x_torch[i]  # 前向传播
        loss = (y_predict - y_torch[i]) ** 2  # 损失函数
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

plt.scatter(x_min_max, y_min_max)
y_final_predict = w.data.item() * x_torch
plt.plot(x_min_max, y_final_predict.numpy())
plt.show()

#### bias 偏置

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

x = np.random.randint(1, 100, 100)
y = np.array([i * 10 + np.random.randint(100) for i in x])
y += 100

x_norm = x / 100.0
y_norm = y / 100.0

x_torch = torch.tensor(x_norm, dtype=torch.float32)
y_torch = torch.tensor(y_norm, dtype=torch.float32)

w = torch.nn.Parameter(torch.tensor(0.1, dtype=torch.float32), requires_grad=True)
b = torch.nn.Parameter(torch.tensor(0.1, dtype=torch.float32), requires_grad=True)

optimizer = torch.optim.SGD([w, b], lr=0.01)  # w -= altha * loss

step = 200
for _ in range(step):
    for i in range(len(x_torch)):
        y_predict = w * x_torch[i] + b  # 前向传播
        loss = (y_predict - y_torch[i]) ** 2  # 损失函数
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        optimizer.zero_grad()  # 梯度清零

plt.scatter(x_norm, y_norm)
y_final_predict = w.data.item() * x_torch + b.data.item()
plt.plot(x_torch, y_final_predict.numpy())
plt.show()

### dataset 和 dataloader

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch
import matplotlib.pyplot as plt
import numpy as np

x = np.random.randint(1, 100, 100)
y = np.array([i * 10 + np.random.randint(100) for i in x])
y += 100

x_norm = x / 100.0
y_norm = y / 100.0

x_torch = torch.tensor(x_norm, dtype=torch.float32)
y_torch = torch.tensor(y_norm, dtype=torch.float32)

dataset = TensorDataset(x_torch, y_torch)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

w = torch.nn.Parameter(torch.tensor(0.1, dtype=torch.float32), requires_grad=True)
b = torch.nn.Parameter(torch.tensor(0.1, dtype=torch.float32), requires_grad=True)

optimizer = torch.optim.SGD([w, b], lr=0.01)  # w -= altha * loss

step = 200
for _ in range(step):
    for x_batch, y_batch in dataloader:
        y_predict = w * x_batch + b  # 前向传播
        loss = ((y_predict - y_batch) ** 2).mean()  # 平均损失函数
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        optimizer.zero_grad()

plt.scatter(x_norm, y_norm)
y_final_predict = (w.data.item() * x_torch + b.data.item()).squeeze()
plt.plot(x_torch, y_final_predict.numpy())
plt.show()

## 分类网络-逻辑回归

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset


def sigmoid(x):
    return 1 / (1 + torch.exp(-x))


x = np.random.randint(-10, 10, 100)
x.sort()

y_0 = np.zeros(50)
y_1 = np.ones(50)
y = np.concatenate([y_0, y_1])

x_torch_tensor = torch.tensor(x, dtype=torch.float32)
y_torch_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(x_torch_tensor, y_torch_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

w = torch.nn.Parameter(torch.rand(1), requires_grad=True)
b = torch.nn.Parameter(torch.rand(1), requires_grad=True)

optimizer = torch.optim.SGD([w, b], lr=0.01)
criterion = torch.nn.MSELoss()

step = 200

for _ in range(step):
    for x_batch, y_batch in dataloader:
        y_predict = sigmoid(w * x_batch + b)
        loss = criterion(y_predict, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

y_final_predict = sigmoid(w.data.item() * x_torch_tensor + b.data.item())
plt.scatter(x_torch_tensor, y_torch_tensor)
plt.plot(x_torch_tensor, y_final_predict.numpy(), c='r')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset


def sigmoid(x):
    return 1 / (1 + torch.exp(-x))


x = np.random.randint(-20, 20, 450)
x.sort()

y_0 = np.zeros(150)
y_1 = np.ones(150)
y_2 = np.zeros(150)
y = np.concatenate([y_0, y_1, y_2])

x_torch_tensor = torch.tensor(x, dtype=torch.float32)
y_torch_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(x_torch_tensor, y_torch_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

w1 = torch.nn.Parameter(torch.rand(1), requires_grad=True)
b1 = torch.nn.Parameter(torch.rand(1), requires_grad=True)
w2 = torch.nn.Parameter(torch.rand(1), requires_grad=True)
b2 = torch.nn.Parameter(torch.rand(1), requires_grad=True)
w3 = torch.nn.Parameter(torch.rand(1), requires_grad=True)
w4 = torch.nn.Parameter(torch.rand(1), requires_grad=True)
b3 = torch.nn.Parameter(torch.rand(1), requires_grad=True)

optimizer = torch.optim.SGD([w1, b1, w2, b2, w3, w4, b3], lr=0.01)
criterion = torch.nn.MSELoss()

step = 1000

for _ in range(step):
    for x_batch, y_batch in dataloader:
        y_predict_1 = sigmoid(w1 * x_batch + b1)
        y_predict_2 = sigmoid(w2 * x_batch + b2)
        y_predict = w3 * y_predict_1 + w4 * y_predict_2 + b3
        loss = criterion(y_predict, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

y_final_predict_1 = sigmoid(w1.data.item() * x_torch_tensor + b1.data.item())
y_final_predict_2 = sigmoid(w2.data.item() * x_torch_tensor + b2.data.item())
y_final_predict = w3.data.item() * y_final_predict_1 + w4.data.item() * y_final_predict_2 + b3.data.item()
plt.scatter(x_torch_tensor, y_torch_tensor)
plt.plot(x_torch_tensor, y_final_predict.detach().numpy(), c='r')
plt.show()

#### pytorch 优化

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

x = np.random.randint(-20, 20, 450)
x.sort()
y_0 = np.zeros(150)
y_1 = np.ones(150)
y_2 = np.zeros(150)
y = np.concatenate([y_0, y_1, y_2])

x_torch_tensor = torch.tensor(x, dtype=torch.float32).view(-1, 1)
y_torch_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

dataset = TensorDataset(x_torch_tensor, y_torch_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)


class SimpleNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.input = torch.nn.Linear(1, 2)  # 输入层
        self.output = torch.nn.Linear(2, 1)  # 输出层

    def forward(self, x):
        y = torch.sigmoid(self.input(x))
        return self.output(y)


model = SimpleNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

step = 1000

for _ in range(step):
    for x_batch, y_batch in dataloader:
        y_predict = model(x_batch)
        loss = criterion(y_predict, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

y_final_predict = model(x_torch_tensor)
plt.scatter(x_torch_tensor, y_torch_tensor)
plt.plot(x_torch_tensor, y_final_predict.detach().numpy(), c='r')
plt.show()

#### 检测梯度

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter

x = np.random.randint(-20, 20, 450)
x.sort()

y_0 = np.zeros(150)
y_1 = np.ones(150)
y_2 = np.zeros(150)
y = np.concatenate([y_0, y_1, y_2])

x_torch_tensor = torch.tensor(x, dtype=torch.float32).view(-1, 1)
y_torch_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

dataset = TensorDataset(x_torch_tensor, y_torch_tensor)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)


class SimpleNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.input = torch.nn.Linear(1, 2)  # 输入层
        self.output = torch.nn.Linear(2, 1)  # 输出层

    def forward(self, x):
        y = torch.sigmoid(self.input(x))
        return self.output(y)


writer = SummaryWriter()
model = SimpleNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

step = 1000
for _ in range(step):
    for x_batch, y_batch in dataloader:
        y_predict = model(x_batch)
        loss = criterion(y_predict, y_batch)
        loss.backward()
        optimizer.step()
        for name, pram in model.named_parameters():
            writer.add_histogram(name, pram.grad, step)
        optimizer.zero_grad()

y_final_predict = model(x_torch_tensor)
plt.scatter(x_torch_tensor, y_torch_tensor)
plt.plot(x_torch_tensor, y_final_predict.detach().numpy(), c='r')
plt.show()

### 分类网络-ReLU

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter

x = np.random.randint(80, 100, 150)
x.sort()

y_0 = np.zeros(50)
y_1 = np.ones(50)
y_2 = np.zeros(50)
y = np.concatenate([y_0, y_1, y_2])

x_torch_tensor = torch.tensor(x, dtype=torch.float32).view(-1, 1)
y_torch_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

dataset = TensorDataset(x_torch_tensor, y_torch_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)


class SimpleNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.input = torch.nn.Linear(1, 2)  # 输入层
        self.output = torch.nn.Linear(2, 1)  # 输出层

    def forward(self, x):
        y = torch.relu(self.input(x))
        return self.output(y)


model = SimpleNetwork()
writer = SummaryWriter()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

step = 1000

for _ in range(step):
    for x_batch, y_batch in dataloader:
        y_predict = model(x_batch)
        loss = criterion(y_predict, y_batch)
        loss.backward()
        optimizer.step()
        for name, pram in model.named_parameters():
            writer.add_histogram(name, pram.grad, step)
        optimizer.zero_grad()

y_final_predict = model(x_torch_tensor)
plt.scatter(x_torch_tensor, y_torch_tensor)
plt.plot(x_torch_tensor, y_final_predict.detach().numpy(), c='r')
plt.show()

### 网络可视化

In [14]:
import torch
from torch.utils.tensorboard import SummaryWriter


class SimpleNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.input = torch.nn.Linear(1, 4)  # 输入层
        self.output = torch.nn.Linear(4, 1)  # 输出层

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = self.output(x)
        return torch.sigmoid(x)


model = SimpleNetwork()
writer = SummaryWriter()

x = torch.randn(100, 1)
writer.add_graph(model, x)
writer.close()